# gdalcubes - Processing Earth Observation using GDAL Image Collection

Managing the files that will be made available during a research study is important for many projects. For example, some researchers cannot publish the data for ethical reasons, while others have problems with the large volume of data used.

To facilitate the reproducibility of these experiments, bdcrrm-api has specific functionalities for file management. These functionalities consider both the production and the reproduction of an experiment.

Therefore, this notebook is used as an example to present the file management capabilities of bdcrrm-api. To this end, this notebook uses the R package gdalcubes to manipulate a local image collection available on the processing machine.

> **Note**: The operations presented here are taken directly from the basic usage example provided by the official gdalcubes documentation. We thank the authors of the excellent package. Thank you.

In [ ]:
set.seed(777)

library(magrittr)
library(gdalcubes)
library(RColorBrewer)

## General definitions

In [ ]:
source("analysis/.pipeline.R")

gdalcubes_options(threads = .pipeline_definitions$threads)

## Processing data cubes

This section presents basic gdalcubes operations, which manipulate a set of scenes available on the processing machine.


**1. Creating an image collection**

In [ ]:
files <- list.files("data/raw_data/L8_Amazon/", recursive = TRUE,
                    full.names = TRUE, pattern = ".tif")

lc8_collection <- create_image_collection(files, format = "L8_SR", 
                                          out_file = "data/derived_data/L8.db")

**2. Creating data cubes view**

In [ ]:
datacube_view <- cube_view(
  extent = lc8_collection,
  dt = "P1Y",
  dx = 1000,
  dy = 1000,
  srs = "EPSG:3857",
  aggregation = "median",
  resampling = "bilinear"
)

**3. Operations**

**3.1 Temporal Aggregation**

In [ ]:
x = raster_cube(lc8_collection, datacube_view) %>%
      select_bands(c("B02", "B03", "B04")) %>%
      reduce_time(c('median(B02)', 'median(B03)', 'median(B04)'))

plot(x, rgb=3:1, zlim=c(0, 1200))

**3.2 Band Math**

In [ ]:
raster_cube(lc8_collection, datacube_view) %>%
    select_bands(c("B04", "B05")) %>%
    apply_pixel(c("(B05-B04) / (B05+B04)"), names="NDVI") %>%
    plot(zlim=c(0,1),  nbreaks=10, col=brewer.pal(9, "YlGn"), key.pos = 1)